# Extract EEG Power Features from Data

** Hardcoded sections for bad channels

- Start with raw data
    - Divide into different frequency bands (each with their own array)
    - Bandpass filter around each frequency
    - Remove SSVEP frequency
    - Remove 16 Hz noise in necessary files
- Import Comfort Data
- Epoch all data
- Compute PSD of all data
- Compute different frequency band ratios
- Export

# Import Libraries

In [1]:
# Import libraries
import mne
import numpy as np
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath("../Functions"))

# Import custom scripts
import import_data
import data_tools
import processing


# Import custom scripts
#from Functions import import_data
#from Functions import data_tools
#from Functions import processing

# Enable interactive plots
%matplotlib qt

# Import EEG Data

In [2]:
# Load list of files to import
files = [  
    "sub-P001_ses-S001_task-T1_run-001_eeg",
    "sub-P002_ses-S001_task-T1_run-001_eeg",
    "sub-P003_ses-S001_task-T1_run-001_eeg",
    "sub-P004_ses-S001_task-T1_run-001_eeg",
    "sub-P005_ses-S001_task-T1_run-001_eeg",
    "sub-P006_ses-S001_task-T1_run-001_eeg",
    "sub-P007_ses-S001_task-T1_run-001_eeg",
    "sub-P008_ses-S001_task-T1_run-001_eeg", 
    "sub-P009_ses-S001_task-T1_run-001_eeg",
    "sub-P010_ses-S001_task-T1_run-001_eeg",  
]

# Get unique subject IDs
subject_ids = [file.split('_')[0] for file in files]
unique_subject_ids = list(set(subject_ids))

# Preallocate eeg_ts, eeg_data, eeg_fs
eeg_ts =   [None] * len(files)
eeg_data = [None] * len(files)
raw_eeg =  [None] * len(files)

ch_names = ["Fz", "F4", "F8", "C3", "Cz", "C4", "T8", "P7", "P3", "P4", "P8", "PO7", "PO8", "O1", "Oz", "O2"]

# HARDCODED FOR BAD CHANNELS
P002_ch_names = ["Fz", "F4", "F8", "C3", "Cz", "C4", "T8", "P7", "P3", "P4", "P8", "PO8", "O1", "Oz", "O2"]
P007_ch_names = ["Fz", "F4", "F8", "C3", "Cz", "C4", "T8", "P3", "P4", "P8", "PO7", "PO8", "O1", "Oz", "O2"]
P008_ch_names = ["Fz", "F4", "F8", "C3", "Cz", "C4", "T8", "P3", "P4", "P8", "PO7", "PO8", "Oz", "O2"]
# END HARDCODED FOR BAD CHANNELS

for f, file in enumerate(files):
    for sub in subject_ids:
        if sub == file.split('_')[0]:

            # HARDCODED FOR BAD CHANNELS
            if sub == "sub-P002":
                ch_names = P002_ch_names
            elif sub == "sub-P007":
                ch_names = P007_ch_names
            elif sub == "sub-P008":
                ch_names = P008_ch_names
            # END HARDCODED FOR BAD CHANNELS

            [eeg_ts[f], eeg_data[f], eeg_fs] = import_data.read_xdf(f"..\\Data\\Pilot2\\EEG\\{sub}\\ses-S001\\eeg\\{file}.xdf", picks=ch_names)

            # Create MNE array
            info = mne.create_info(ch_names, eeg_fs, ch_types = 'eeg')  # Create info properties
            raw_eeg[f] = mne.io.RawArray(eeg_data[f], info = info)            

            # Set standard channel montage
            raw_eeg[f].set_montage('standard_1020')

            #reset channel names
            # HARDCODED FOR BAD CHANNELS
            ch_names = ["Fz", "F4", "F8", "C3", "Cz", "C4", "T8", "P7", "P3", "P4", "P8", "PO7", "PO8", "O1", "Oz", "O2"]
            # END HARDCODED FOR BAD CHANNELS

Creating RawArray with float64 data, n_channels=16, n_times=207136
    Range : 0 ... 207135 =      0.000 ...   809.121 secs
Ready.
Creating RawArray with float64 data, n_channels=15, n_times=194944
    Range : 0 ... 194943 =      0.000 ...   761.496 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=199168
    Range : 0 ... 199167 =      0.000 ...   777.996 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=201344
    Range : 0 ... 201343 =      0.000 ...   786.496 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=195456
    Range : 0 ... 195455 =      0.000 ...   763.496 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=196992
    Range : 0 ... 196991 =      0.000 ...   769.496 secs
Ready.
Creating RawArray with float64 data, n_channels=15, n_times=200960
    Range : 0 ... 200959 =      0.000 ...   784.996 secs
Ready.
Creating RawArray with float64 data, n_channels=14, n_times=190080
    Range : 0 ..

# Import Comfort Data

In [3]:
# Initialize list of dictionaries, one for each file
comfort_data = [None] * len(files)
comfort_data_dicts = [{} for _ in range(len(files))]  # List of empty dictionaries

for f, file in enumerate(files):
    for sub in subject_ids:
        if sub == file.split('_')[0]:
            subject_num = sub.split('-')[1]  # Extract subject number from ID
            
            # Import comfort data
            comfort_data[f] = pd.read_csv(f"..\\Data\\Pilot2\\Bracket\\{subject_num}\\{subject_num}_single_absolute.csv") 
            comfort_data[f]['Epoch'] -= 1  # Adjust epoch index to start from 0
            
            # Group by contrast and size
            grouped = comfort_data[f].groupby(["Contrast", "Size"])
            
            # Create dictionary of comfort scores for each contrast/size
            stim_comfort_dict = {}
            for (contrast, size), group in grouped:
                key = f"Contrast{contrast}Size{size}"
                values = group["Comfort_Score"].tolist()
                stim_comfort_dict[key] = values
            
            # Store in the list of dictionaries
            comfort_data_dicts[f] = stim_comfort_dict

# Divide into different signal arrays to apply filters

In [4]:
# Preallocate lists for filtered data
delta_raw = [None] * len(files)
theta_raw = [None] * len(files)
alpha_raw = [None] * len(files)
beta_raw  = [None] * len(files)

# Loop through each file and corresponding raw EEG
for f, file in enumerate(files):
    subject_id = file.split('_')[0]
    print(f"Processing file {f+1}/{len(files)}: {file} for subject {subject_id}")

    # Assign subject-specific channel names
    # HARDCODED FOR BAD CHANNELS
    if subject_id == "sub-P002":
        ch_names = P002_ch_names
    elif subject_id == "sub-P007":
        ch_names = P007_ch_names
    elif subject_id == "sub-P008":
        ch_names = P008_ch_names
    # END HARDCODED FOR BAD CHANNELS

    raw = raw_eeg[f]

    # Filter EEG data by frequency bands
    delta_raw[f] = raw.copy().filter(l_freq=1, h_freq=4, picks=ch_names)
    theta_raw[f] = raw.copy().filter(l_freq=4, h_freq=8, picks=ch_names)
    alpha_raw[f] = raw.copy().filter(l_freq=8, h_freq=12, picks=ch_names)
    beta_raw[f]  = raw.copy().filter(l_freq=12, h_freq=30, picks=ch_names)

    # Apply notch filters
    alpha_raw[f] = alpha_raw[f].copy().notch_filter(freqs=10, picks=ch_names)
    beta_raw[f]  = beta_raw[f].copy().notch_filter(freqs=20, picks=ch_names)

    #reset channel names
    # HARDCODED FOR BAD CHANNELS
    ch_names = ["Fz", "F4", "F8", "C3", "Cz", "C4", "T8", "P7", "P3", "P4", "P8", "PO7", "PO8", "O1", "Oz", "O2"]
    # END HARDCODED FOR BAD CHANNELS

Processing file 1/10: sub-P001_ses-S001_task-T1_run-001_eeg for subject sub-P001
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 845 samples (3.301 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition b

# Notch filter 16 Hz noise if necessary
- Several files have a bunch of 16 Hz noise (for pilot_v2: P001, P003, P007)

In [5]:
for f, file in enumerate(files):
    for sub in subject_ids:
        if sub == file.split('_')[0]:
            if sub == "sub-P001" or sub == "sub-P003":
                beta_raw[f] = beta_raw[f].copy().notch_filter(freqs=16, picks=ch_names)
            # HARDCODED FOR BAD CHANNELS
            elif sub == "sub-P007":
                beta_raw[f] = beta_raw[f].copy().notch_filter(freqs=16, picks=P007_ch_names)
            # END HARDCODED FOR BAD CHANNELS

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 15 - 17 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.46
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 15.21 Hz)
- Upper passband edge: 16.54 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 16.79 Hz)
- Filter length: 1691 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 15 - 17 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.46
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 15.21 Hz)
- Upper passband e

# Fix P001 labels

In [6]:
[marker_ts_P001, markers_P001] = import_data.read_xdf_unity_markers(f"..\\Data\\Pilot2\\EEG\\sub-P001\\ses-S001\\eeg\\sub-P001_ses-S001_task-T1_run-001_eeg.xdf")

for temp in markers_P001:
    if temp[0] == "stimulus ended, getting score":
        temp[0] = "bleh" 

epoch_end_P001 = "bleh"

# Epoch data for each band

For pilot version 2, P001 has labels that need different processing

In [7]:
list_of_events = [] # same for all files

# Initialize lists for markers and marker timestamps
markers = [None] * len(files)
marker_ts = [None] * len(files)

# Initialize lists for epochs and EEG data for each frequency band
delta_events_epochs, delta_eeg_epochs, delta_epochs_organized = ([None] * len(files) for _ in range(3))
theta_events_epochs, theta_eeg_epochs, theta_epochs_organized = ([None] * len(files) for _ in range(3))
alpha_events_epochs, alpha_eeg_epochs, alpha_epochs_organized = ([None] * len(files) for _ in range(3))
beta_events_epochs, beta_eeg_epochs, beta_epochs_organized    = ([None] * len(files) for _ in range(3))

# Create a list of unique events
for x in range(4):
    for y in range(3):
        list_of_events.append(f"Contrast{x+1}Size{y+1}")

epoch_end = "getting score"

# Create a dict of stimuli using the unique events
dict_of_stimuli = {i: event for i, event in enumerate(list_of_events)}

for f, file in enumerate(files):
    for sub in subject_ids:
        if sub == file.split('_')[0]:
            if sub == "sub-P001": # Using special labels for P001
                [delta_events_epochs[0], delta_eeg_epochs[0]] = data_tools.create_epochs(eeg_data = delta_raw[0].get_data(), eeg_ts = eeg_ts[0], markers = markers_P001, markers_ts = marker_ts_P001, events = list_of_events, epoch_end = epoch_end_P001)
                [theta_events_epochs[0], theta_eeg_epochs[0]] = data_tools.create_epochs(eeg_data = theta_raw[0].get_data(), eeg_ts = eeg_ts[0], markers = markers_P001, markers_ts = marker_ts_P001, events = list_of_events, epoch_end = epoch_end_P001)
                [alpha_events_epochs[0], alpha_eeg_epochs[0]] = data_tools.create_epochs(eeg_data = alpha_raw[0].get_data(), eeg_ts = eeg_ts[0], markers = markers_P001, markers_ts = marker_ts_P001, events = list_of_events, epoch_end = epoch_end_P001)
                [beta_events_epochs[0],  beta_eeg_epochs[0]]  = data_tools.create_epochs(eeg_data = beta_raw[0].get_data(),  eeg_ts = eeg_ts[0], markers = markers_P001, markers_ts = marker_ts_P001, events = list_of_events, epoch_end = epoch_end_P001)

            else:  # For all other subjects
                [marker_ts[f], markers[f]] = import_data.read_xdf_unity_markers(f"..\\Data\\Pilot2\\EEG\\{sub}\\ses-S001\\eeg\\{file}.xdf") # Import marker

                [delta_events_epochs[f], delta_eeg_epochs[f]] = data_tools.create_epochs(eeg_data = delta_raw[f].get_data(), eeg_ts = eeg_ts[f], markers = markers[f], markers_ts = marker_ts[f], events = list_of_events, epoch_end = epoch_end)
                [theta_events_epochs[f], theta_eeg_epochs[f]] = data_tools.create_epochs(eeg_data = theta_raw[f].get_data(), eeg_ts = eeg_ts[f], markers = markers[f], markers_ts = marker_ts[f], events = list_of_events, epoch_end = epoch_end)
                [alpha_events_epochs[f], alpha_eeg_epochs[f]] = data_tools.create_epochs(eeg_data = alpha_raw[f].get_data(), eeg_ts = eeg_ts[f], markers = markers[f], markers_ts = marker_ts[f], events = list_of_events, epoch_end = epoch_end)
                [beta_events_epochs[f],  beta_eeg_epochs[f]]  = data_tools.create_epochs(eeg_data = beta_raw[f].get_data(),  eeg_ts = eeg_ts[f], markers = markers[f], markers_ts = marker_ts[f], events = list_of_events, epoch_end = epoch_end)

# Organize epochs by stimuli and frequency
for f, file in enumerate(files):
    delta_epochs_organized[f] = data_tools.epochs_stim(eeg_epochs = delta_eeg_epochs[f], labels = delta_events_epochs[f], stimuli = dict_of_stimuli)
    theta_epochs_organized[f] = data_tools.epochs_stim(eeg_epochs = theta_eeg_epochs[f], labels = theta_events_epochs[f], stimuli = dict_of_stimuli)
    alpha_epochs_organized[f] = data_tools.epochs_stim(eeg_epochs = alpha_eeg_epochs[f], labels = alpha_events_epochs[f], stimuli = dict_of_stimuli)
    beta_epochs_organized[f]  = data_tools.epochs_stim(eeg_epochs = beta_eeg_epochs[f],  labels = beta_events_epochs[f],  stimuli = dict_of_stimuli)

# Make organized epochs into dictionaries

In [8]:
# Define dictionaries for each frequency band and file
delta_dicts = [{} for _ in range(len(files))] 
theta_dicts = [{} for _ in range(len(files))]
alpha_dicts = [{} for _ in range(len(files))]
beta_dicts  = [{} for _ in range(len(files))]

for f, file in enumerate(files):
    delta_dicts[f] = {stim_label: np.array(delta_epochs_organized[f][stim_idx]) for stim_idx, stim_label in dict_of_stimuli.items()}
    theta_dicts[f] = {stim_label: np.array(theta_epochs_organized[f][stim_idx]) for stim_idx, stim_label in dict_of_stimuli.items()}
    alpha_dicts[f] = {stim_label: np.array(alpha_epochs_organized[f][stim_idx]) for stim_idx, stim_label in dict_of_stimuli.items()}
    beta_dicts[f]  = {stim_label: np.array(beta_epochs_organized[f][stim_idx])  for stim_idx, stim_label in dict_of_stimuli.items()}

# Compute PSD for each band

In [9]:
window_size = 5

delta_eeg_f, delta_eeg_pxx = ([None] * len(files) for _ in range(2))
theta_eeg_f, theta_eeg_pxx = ([None] * len(files) for _ in range(2))
alpha_eeg_f, alpha_eeg_pxx = ([None] * len(files) for _ in range(2))
beta_eeg_f,  beta_eeg_pxx  = ([None] * len(files) for _ in range(2))

for f, file in enumerate(files):
    # Calculate power spectral density for each frequency band
    delta_eeg_f[f], delta_eeg_pxx[f] = processing.psd(delta_dicts[f], eeg_fs, window_size)
    theta_eeg_f[f], theta_eeg_pxx[f] = processing.psd(theta_dicts[f], eeg_fs, window_size)
    alpha_eeg_f[f], alpha_eeg_pxx[f] = processing.psd(alpha_dicts[f], eeg_fs, window_size)
    beta_eeg_f[f],  beta_eeg_pxx[f]  = processing.psd(beta_dicts[f],  eeg_fs, window_size)

#verify the size of the data
print(f"Delta: {len(delta_eeg_pxx[1])} stimuli, {len(delta_eeg_pxx[1]['Contrast1Size1'])} epochs, {len(delta_eeg_pxx[1]['Contrast1Size1'][0])} channels, {len(delta_eeg_pxx[1]['Contrast1Size1'][0][0])} frequencies")

c:\Users\admin\miniconda3\envs\bessy\Lib\site-packages\scipy\signal\_spectral_py.py:790: UserWarning: nperseg = 1280 is greater than input length  = 1279, using nperseg = 1279
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Delta: 12 stimuli, 3 epochs, 15 channels, 641 frequencies


# Get the mean PSD for each epoch (for each band)

Shape (for each file) was: (num_stimuli, num_epochs, num_channels, num_pxx) 
- Typically (12, N, 16, 641)
- For 10 files: (10, 12, N, 16, 641)

Now, want to average over the pxx to get a single PSD value for each epoch
- Shape will be: (num_stimuli, num_epochs, num_channels)
- When considering files: (num-files, num_stimuli, num_epochs, num_channels)

In [10]:
# Define dictionaries for each frequency band and file
delta_mean_power_per_epoch = [{} for _ in range(len(files))]
theta_mean_power_per_epoch = [{} for _ in range(len(files))]    
alpha_mean_power_per_epoch = [{} for _ in range(len(files))]
beta_mean_power_per_epoch  = [{} for _ in range(len(files))]

# Calculate mean power per epoch for each frequency band and file
for f, file in enumerate(files):
    for stim_label, pxx_list in delta_eeg_pxx[f].items():
        delta_mean_power_per_epoch[f][stim_label] = [np.mean(pxx, axis = 1) for pxx in pxx_list]

    for stim_label, pxx_list in theta_eeg_pxx[f].items():
        theta_mean_power_per_epoch[f][stim_label] = [np.mean(pxx, axis = 1) for pxx in pxx_list]

    for stim_label, pxx_list in alpha_eeg_pxx[f].items():
        alpha_mean_power_per_epoch[f][stim_label] = [np.mean(pxx, axis = 1) for pxx in pxx_list]

    for stim_label, pxx_list in beta_eeg_pxx[f].items():
        beta_mean_power_per_epoch[f][stim_label]  = [np.mean(pxx, axis = 1) for pxx in pxx_list]

# Calculate Ratios

In [11]:
# allocate empty dictionaries for ratios
alpha_beta_ratio =                [{} for _ in range(len(files))]
theta_alpha_ratio =               [{} for _ in range(len(files))]
theta_beta_ratio =                [{} for _ in range(len(files))]
theta_alpha_sum =                 [{} for _ in range(len(files))]
theta_alpha_by_beta_ratio =       [{} for _ in range(len(files))]
theta_alpha_beta_sum =            [{} for _ in range(len(files))]
alpha_theta_by_alpha_beta_ratio = [{} for _ in range(len(files))]
theta_by_alpha_beta_ratio =       [{} for _ in range(len(files))]

for f, file in enumerate(files):
    #alpha/beta ratio
    for stim_label in alpha_mean_power_per_epoch[f].keys(): #shape is (12, N, 16)
        alpha_beta_ratio[f][stim_label] = np.array(alpha_mean_power_per_epoch[f][stim_label]) / np.array(beta_mean_power_per_epoch[f][stim_label])

    #theta/alpha ratio
    for stim_label in theta_mean_power_per_epoch[f].keys(): 
        theta_alpha_ratio[f][stim_label] = np.array(theta_mean_power_per_epoch[f][stim_label]) / np.array(alpha_mean_power_per_epoch[f][stim_label])

    #theta/beta ratio
    for stim_label in theta_mean_power_per_epoch[f].keys(): 
        theta_beta_ratio[f][stim_label] = np.array(theta_mean_power_per_epoch[f][stim_label]) / np.array(beta_mean_power_per_epoch[f][stim_label])

    #theta + alpha
    for stim_label in theta_mean_power_per_epoch[f].keys(): 
        theta_alpha_sum[f][stim_label] = np.array(theta_mean_power_per_epoch[f][stim_label]) + np.array(alpha_mean_power_per_epoch[f][stim_label])

    #theta + alpha/ beta
    for stim_label in theta_mean_power_per_epoch[f].keys(): 
        theta_alpha_by_beta_ratio[f][stim_label] = np.array(theta_alpha_sum[f][stim_label]) / np.array(beta_mean_power_per_epoch[f][stim_label])

    # theta + alpha + beta
    for stim_label in theta_mean_power_per_epoch[f].keys(): 
        theta_alpha_beta_sum[f][stim_label] = np.array(theta_mean_power_per_epoch[f][stim_label]) + np.array(alpha_mean_power_per_epoch[f][stim_label]) + np.array(beta_mean_power_per_epoch[f][stim_label])

    #(alpha + theta)/(alpha + beta)
    for stim_label in theta_mean_power_per_epoch[f].keys():
        alpha_theta_by_alpha_beta_ratio[f][stim_label] = np.array(theta_alpha_sum[f][stim_label]) / (np.array(alpha_mean_power_per_epoch[f][stim_label]) + np.array(beta_mean_power_per_epoch[f][stim_label]))

    #theta/(alpha + beta)
    for stim_label in theta_mean_power_per_epoch[f].keys():
        theta_by_alpha_beta_ratio[f][stim_label] = np.array(theta_mean_power_per_epoch[f][stim_label]) / (np.array(alpha_mean_power_per_epoch[f][stim_label]) + np.array(beta_mean_power_per_epoch[f][stim_label]))

# Export

In [12]:
export = False

if export:
    records = []

    # HARDCODED FOR BAD CHANNELS
    ch_names_temp = ch_names
    # END HARDCODED FOR BAD CHANNELS

    for f, file in enumerate(files):
        participant_id = file.split('_')[0]
        comfort_data_dict = comfort_data_dicts[f]

        all_band_data = {
            'Delta': delta_mean_power_per_epoch[f],
            'Theta': theta_mean_power_per_epoch[f],
            'Alpha': alpha_mean_power_per_epoch[f],
            'Beta': beta_mean_power_per_epoch[f],
            'Alpha/Beta': alpha_beta_ratio[f],
            'Theta/Alpha': theta_alpha_ratio[f],
            'Theta/Beta': theta_beta_ratio[f],
            'Theta_+_Alpha': theta_alpha_sum[f],
            'Theta_+_Alpha/Beta': theta_alpha_by_beta_ratio[f],
            'Theta_+_Alpha_+_Beta': theta_alpha_beta_sum[f],
            'Alpha_+_Theta/Alpha_+_Beta': alpha_theta_by_alpha_beta_ratio[f],
            'Theta/Alpha_+_Beta': theta_by_alpha_beta_ratio[f]
        }

        # Iterate over stimuli present in the band data
        for stim_label in list(all_band_data.values())[0].keys():
            n_epochs = len(all_band_data['Delta'][stim_label])  # Safe number of available epochs
       
            for epoch_idx in range(n_epochs):
                # Try to get comfort score if available
                if stim_label in comfort_data_dict and epoch_idx < len(comfort_data_dict[stim_label]):
                    comfort_score = comfort_data_dict[stim_label][epoch_idx]
                else:
                    comfort_score = np.nan  # Missing value (this should not happen)
                    print("ERROR")

                row = {
                    'Participant': participant_id,
                    'Stimulus': stim_label,
                    'Epoch': epoch_idx,
                    'Comfort Score': comfort_score
                }

                for band_name, band_dict in all_band_data.items():
                    pxx = band_dict[stim_label][epoch_idx]  # shape: (channels,)

                    # HARDCODED FOR BAD CHANNELS
                    if participant_id == "sub-P002":
                        ch_names_temp = P002_ch_names
                    elif participant_id == "sub-P007":
                        ch_names_temp = P007_ch_names
                    elif participant_id == "sub-P008":
                        ch_names_temp = P008_ch_names
                    # END HARDCODED FOR BAD CHANNELS

                    for ch_idx, ch_name in enumerate(ch_names):
                        col_name = f"{ch_name}_{band_name.lower()}"

                        # HARDCODED FOR BAD CHANNELS
                        if ch_name not in ch_names_temp: # Skip channels not in the subject's configuration
                            row[col_name] = np.nan  # Missing value for this channel
                        # END HARDCODED FOR BAD CHANNELS

                        else:
                            ch_idx_temp = ch_names_temp.index(ch_name)
                            row[col_name] = pxx[ch_idx_temp]

                records.append(row)

                # HARDCODED FOR BAD CHANNELS
                ch_names_temp = ch_names
                # END HARDCODED FOR BAD CHANNELS

    # Combine all into one DataFrame and save
    df_all = pd.DataFrame(records)
    df_all.to_csv("all_participants_eeg_band_power.csv", index=False)